In [1]:
import pandas as pd

In [2]:
emdat_path = "/home/nissim/Documents/dev/arg-inundaciones/data/public_emdat_custom_request_2025-06-26_fe5041a7-dc26-4391-aef0-f49e5e5d8657.xlsx"
emdat = pd.read_excel(emdat_path)

In [3]:
print(emdat.columns.to_list())

['DisNo.', 'Historic', 'Classification Key', 'Disaster Group', 'Disaster Subgroup', 'Disaster Type', 'Disaster Subtype', 'External IDs', 'Event Name', 'ISO', 'Country', 'Subregion', 'Region', 'Location', 'Origin', 'Associated Types', 'OFDA/BHA Response', 'Appeal', 'Declaration', "AID Contribution ('000 US$)", 'Magnitude', 'Magnitude Scale', 'Latitude', 'Longitude', 'River Basin', 'Start Year', 'Start Month', 'Start Day', 'End Year', 'End Month', 'End Day', 'Total Deaths', 'No. Injured', 'No. Affected', 'No. Homeless', 'Total Affected', "Reconstruction Costs ('000 US$)", "Reconstruction Costs, Adjusted ('000 US$)", "Insured Damage ('000 US$)", "Insured Damage, Adjusted ('000 US$)", "Total Damage ('000 US$)", "Total Damage, Adjusted ('000 US$)", 'CPI', 'Admin Units', 'Entry Date', 'Last Update']


In [4]:
# Check the data type of the column
print("Column dtype:", emdat["Admin Units"].dtype)
print("Column type:", type(emdat["Admin Units"]))

# Check the type of individual elements
print("\nFirst element type:", type(emdat["Admin Units"].iloc[0]))
print("Second element type:", type(emdat["Admin Units"].iloc[1]))

# Look at the actual content of a few rows
print("\nFirst row content:")
print(emdat["Admin Units"].iloc[0])
print("\nSecond row content:")
print(emdat["Admin Units"].iloc[1])

# Check if it's a string that needs parsing
if isinstance(emdat["Admin Units"].iloc[0], str):
    print("\nIt's a string - might need JSON parsing")
    import json

    try:
        parsed = json.loads(emdat["Admin Units"].iloc[0])
        print("Successfully parsed as JSON:", type(parsed))
    except:
        print("Not valid JSON")
elif isinstance(emdat["Admin Units"].iloc[0], list):
    print("\nIt's already a list")
    print("List element types:", [type(item) for item in emdat["Admin Units"].iloc[0]])

Column dtype: object
Column type: <class 'pandas.core.series.Series'>

First element type: <class 'str'>
Second element type: <class 'str'>

First row content:
[{"adm1_code":431,"adm1_name":"Catamarca"},{"adm1_code":434,"adm1_name":"Cordoba"},{"adm1_code":438,"adm1_name":"Jujuy"},{"adm1_code":440,"adm1_name":"La Rioja"},{"adm1_code":445,"adm1_name":"Salta"},{"adm1_code":450,"adm1_name":"Santiago Del Estero"},{"adm1_code":452,"adm1_name":"Tucuman"}]

Second row content:
[{"adm1_code":430,"adm1_name":"Buenos Aires D.f."},{"adm1_code":434,"adm1_name":"Cordoba"},{"adm1_code":439,"adm1_name":"La Pampa"},{"adm2_code":4386,"adm2_name":"Avellaneda"},{"adm2_code":4395,"adm2_name":"Berisso"},{"adm2_code":4445,"adm2_name":"Lanus"},{"adm2_code":4477,"adm2_name":"Quilmes"},{"adm2_code":82738,"adm2_name":"San Miguel"},{"adm2_code":190525,"adm2_name":"San  Fernando"},{"adm2_code":4631,"adm2_name":"Parana"},{"adm2_code":4836,"adm2_name":"Rosario"}]

It's a string - might need JSON parsing
Successfully

In [5]:
import json
import pandas as pd


def extract_admin_names(admin_units_str):
    """
    Extract adm1 and adm2 names from a JSON string of admin units.
    Returns (adm1_names, adm2_names) as lists.
    """
    try:
        parsed = json.loads(admin_units_str)
        adm1_names = []
        adm2_names = []

        for unit in parsed:
            if "adm1_name" in unit:
                adm1_names.append(unit["adm1_name"])
            if "adm2_name" in unit:
                adm2_names.append(unit["adm2_name"])

        return adm1_names, adm2_names
    except:
        # Return empty lists if parsing fails
        return [], []


# Apply the extraction function to create new columns
emdat[["adm1_names", "adm2_names"]] = emdat["Admin Units"].apply(
    lambda x: pd.Series(extract_admin_names(x))
)

# Display the results
print("=== EXTRACTION RESULTS ===")
print(f"Total rows processed: {len(emdat)}")
print(f"Rows with adm1 names: {sum(emdat['adm1_names'].apply(len) > 0)}")
print(f"Rows with adm2 names: {sum(emdat['adm2_names'].apply(len) > 0)}")

# Show a few examples
print("\n=== SAMPLE RESULTS ===")
for i in range(min(5, len(emdat))):
    print(f"Row {i}:")
    print(f"  adm1_names: {emdat['adm1_names'].iloc[i]}")
    print(f"  adm2_names: {emdat['adm2_names'].iloc[i]}")
    print()

=== EXTRACTION RESULTS ===
Total rows processed: 43
Rows with adm1 names: 27
Rows with adm2 names: 17

=== SAMPLE RESULTS ===
Row 0:
  adm1_names: ['Catamarca', 'Cordoba', 'Jujuy', 'La Rioja', 'Salta', 'Santiago Del Estero', 'Tucuman']
  adm2_names: []

Row 1:
  adm1_names: ['Buenos Aires D.f.', 'Cordoba', 'La Pampa']
  adm2_names: ['Avellaneda', 'Berisso', 'Lanus', 'Quilmes', 'San Miguel', 'San  Fernando', 'Parana', 'Rosario']

Row 2:
  adm1_names: ['Buenos Aires', 'Cordoba', 'La Pampa', 'Santa Fe']
  adm2_names: []

Row 3:
  adm1_names: []
  adm2_names: ['Iriondo']

Row 4:
  adm1_names: ['Buenos Aires']
  adm2_names: []



In [6]:
from shapely.geometry import box
from utils.pygeoboundaries.main import get_area_of_interest_by_names


def get_flood_bounding_box(emdat_row, country_iso3):
    """
    Get bounding box for a flood event based on admin unit names.

    Args:
        emdat_row: Single row from emdat dataframe with adm1_names and adm2_names columns
        country_iso3: ISO3 country code for filtering

    Returns:
        bounding_box: shapely geometry representing the bounding box
        bbox_dict: dictionary with bounding box coordinates
    """

    # Check if we have adm2 names (prefer adm2 over adm1 for more precision)
    if len(emdat_row["adm2_names"]) > 0:
        adm2_names = emdat_row["adm2_names"]
        print(f"Filtered by {len(adm2_names)} adm2 names: {adm2_names}")

        # Get bounding box using geoBoundaries
        bbox_dict = get_area_of_interest_by_names(
            unit_names=adm2_names, adm_level="ADM2", country_iso3=country_iso3
        )

    elif len(emdat_row["adm1_names"]) > 0:
        adm1_names = emdat_row["adm1_names"]
        print(f"Filtered by {len(adm1_names)} adm1 names: {adm1_names}")

        # Get bounding box using geoBoundaries
        bbox_dict = get_area_of_interest_by_names(
            unit_names=adm1_names, adm_level="ADM1", country_iso3=country_iso3
        )

    else:
        print("No admin names found!")
        return None, None

    if bbox_dict is None:
        print("No matching administrative units found!")
        return None, None

    # Convert dictionary to shapely geometry
    bbox_geometry = box(
        bbox_dict["min_lon"],
        bbox_dict["min_lat"],
        bbox_dict["max_lon"],
        bbox_dict["max_lat"],
    )

    print(f"Bounding box coordinates: {bbox_dict}")
    print(f"Bounding box area: {bbox_geometry.area:.6f}")

    return bbox_geometry, bbox_dict


# Example usage for a single row
# Assuming you have the country ISO3 code (e.g., 'ARG' for Argentina)
country_iso3 = "ARG"  # You'll need to get this from your emdat data

# Test with row 1 (which has both adm1 and adm2 names)
test_row = emdat.iloc[1]
print("=== TESTING ROW 1 ===")
print(f"adm1_names: {test_row['adm1_names']}")
print(f"adm2_names: {test_row['adm2_names']}")

bbox, bbox_dict = get_flood_bounding_box(test_row, country_iso3)

if bbox is not None:
    print(f"\nBounding box geometry: {bbox}")

    # You can also get the coordinates for satellite imagery search
    bounds = bbox.bounds
    print(f"Min/Max coordinates: {bounds}")

=== TESTING ROW 1 ===
adm1_names: ['Buenos Aires D.f.', 'Cordoba', 'La Pampa']
adm2_names: ['Avellaneda', 'Berisso', 'Lanus', 'Quilmes', 'San Miguel', 'San  Fernando', 'Parana', 'Rosario']
Filtered by 8 adm2 names: ['Avellaneda', 'Berisso', 'Lanus', 'Quilmes', 'San Miguel', 'San  Fernando', 'Parana', 'Rosario']
Bounding box coordinates: {'min_lat': -39.92531, 'max_lat': -27.468575, 'min_lon': -67.8722, 'max_lon': -56.9436}
Bounding box area: 136.134674

Bounding box geometry: POLYGON ((-56.9436 -39.92531, -56.9436 -27.468575, -67.8722 -27.468575, -67.8722 -39.92531, -56.9436 -39.92531))
Min/Max coordinates: (-67.8722, -39.92531, -56.9436, -27.468575)


In [7]:
import pandas as pd
from datetime import datetime, timedelta


# Create end_date column from End Year, End Month, End Day
def create_end_date(row):
    """
    Create end_date in YYYY-MM-DD format from End Year, End Month, End Day columns.
    Handles missing values by returning None.
    """
    try:
        year = row["End Year"]
        month = row["End Month"]
        day = row["End Day"]

        # Check if all values are present and valid
        if pd.notna(year) and pd.notna(month) and pd.notna(day):
            # Convert to integers and create date
            year = int(year)
            month = int(month)
            day = int(day)

            # Validate date
            date_obj = datetime(year, month, day)
            return date_obj.strftime("%Y-%m-%d")
        else:
            return None
    except (ValueError, TypeError):
        return None


# Apply the function to create end_date column
emdat["end_date"] = emdat.apply(create_end_date, axis=1)


# Create datetime range for satellite imagery search
def create_datetime_range(end_date_str):
    """
    Create datetime range starting from end_date and ending 2 days after.
    Format: "YYYY-MM-DD/YYYY-MM-DD"
    """
    if end_date_str is None:
        return None

    try:
        end_date = datetime.strptime(end_date_str, "%Y-%m-%d")
        start_date = end_date
        end_date_plus_2 = end_date + timedelta(days=2)

        return (
            f"{start_date.strftime('%Y-%m-%d')}/{end_date_plus_2.strftime('%Y-%m-%d')}"
        )
    except ValueError:
        return None


# Create datetime_range column
emdat["datetime_range"] = emdat["end_date"].apply(create_datetime_range)

# Display results
print("=== DATE PROCESSING RESULTS ===")
print(f"Total rows: {len(emdat)}")
print(f"Rows with valid end_date: {emdat['end_date'].notna().sum()}")
print(f"Rows with valid datetime_range: {emdat['datetime_range'].notna().sum()}")

# Show sample results
print("\n=== SAMPLE RESULTS ===")
sample_cols = ["End Year", "End Month", "End Day", "end_date", "datetime_range"]
print(emdat[sample_cols].head(10))

# Example usage for a specific row
test_row = emdat.iloc[1]
if test_row["datetime_range"] is not None:
    print(f"\nExample datetime range for row 1: {test_row['datetime_range']}")
    print(
        f"This represents: {test_row['datetime_range'].split('/')[0]} to {test_row['datetime_range'].split('/')[1]}"
    )

=== DATE PROCESSING RESULTS ===
Total rows: 43
Rows with valid end_date: 41
Rows with valid datetime_range: 41

=== SAMPLE RESULTS ===
   End Year  End Month  End Day    end_date         datetime_range
0      2000          3     17.0  2000-03-17  2000-03-17/2000-03-19
1      2000          5     15.0  2000-05-15  2000-05-15/2000-05-17
2      2000         11     16.0  2000-11-16  2000-11-16/2000-11-18
3      2000         11     22.0  2000-11-22  2000-11-22/2000-11-24
4      2001          3     21.0  2001-03-21  2001-03-21/2001-03-23
5      2001          7      NaN        None                   None
6      2001         12      1.0  2001-12-01  2001-12-01/2001-12-03
7      2002         11      2.0  2002-11-02  2002-11-02/2002-11-04
8      2003          2     10.0  2003-02-10  2003-02-10/2003-02-12
9      2003          5     10.0  2003-05-10  2003-05-10/2003-05-12

Example datetime range for row 1: 2000-05-15/2000-05-17
This represents: 2000-05-15 to 2000-05-17


## STAC query
Filter for Sentinel 1 and/or 2 images that are 1) within two days of the flood event, 2) below 20% cloud cover, 3) within the bounding box (duh), 4) ???.

In [8]:
# Import required libraries
import pandas as pd
import planetary_computer
import pystac_client
import panel as pn

# Enable Panel for interactive visualizations
pn.extension()

In [9]:
# Step 1: Connect to Planetary Computer Catalog
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,  # Automatically signs requests
)

In [12]:
import pandas as pd
from tqdm import tqdm

# Initialize results storage
results = []

print("=== SEARCHING SENTINEL-1 RTC FOR ALL EMDAT ROWS ===")

# Process each row
for idx, row in tqdm(emdat.iterrows(), total=len(emdat), desc="Processing rows"):
    try:
        # Get bounding box for this row
        bbox_geometry, bbox_dict = get_flood_bounding_box(row, "ARG")

        if bbox_geometry is None:
            results.append(
                {
                    "row_index": idx,
                    "event_name": row.get("Event Name", "Unknown"),
                    "end_date": row.get("end_date", None),
                    "datetime_range": row.get("datetime_range", None),
                    "bbox_available": False,
                    "items_found": 0,
                    "error": "No bounding box available",
                }
            )
            continue

        # Convert shapely geometry bounds to bbox format
        bounds = bbox_geometry.bounds
        bbox = [bounds[0], bounds[1], bounds[2], bounds[3]]

        # Get datetime range
        datetime_range = row.get("datetime_range")

        if datetime_range is None:
            results.append(
                {
                    "row_index": idx,
                    "event_name": row.get("Event Name", "Unknown"),
                    "end_date": row.get("end_date", None),
                    "datetime_range": None,
                    "bbox_available": True,
                    "items_found": 0,
                    "error": "No valid date range",
                }
            )
            continue

        # Search for Sentinel-1 RTC data
        search = catalog.search(
            collections=["sentinel-1-rtc"],
            bbox=bbox,
            datetime=datetime_range,
            query={
                "sar:instrument_mode": {"eq": "IW"},
                "sar:frequency_band": {"eq": "C"},
                "sar:polarizations": {"in": [["VV"], ["VH"], ["VV", "VH"]]},
            },
        )

        items = search.item_collection()

        # Store results
        results.append(
            {
                "row_index": idx,
                "event_name": row.get("Event Name", "Unknown"),
                "end_date": row.get("end_date", None),
                "datetime_range": datetime_range,
                "bbox_available": True,
                "items_found": len(items),
                "bbox_coords": bbox,
                "error": None,
            }
        )

    except Exception as e:
        results.append(
            {
                "row_index": idx,
                "event_name": row.get("Event Name", "Unknown"),
                "end_date": row.get("end_date", None),
                "datetime_range": row.get("datetime_range", None),
                "bbox_available": False,
                "items_found": 0,
                "error": str(e),
            }
        )

# Convert results to DataFrame
results_df = pd.DataFrame(results)

# Summary statistics
print(f"\nTotal rows: {len(results_df)}")
print(f"Rows with imagery: {(results_df['items_found'] > 0).sum()}")
print(f"Total items found: {results_df['items_found'].sum()}")

# Show rows with imagery
rows_with_imagery = results_df[results_df["items_found"] > 0]
if len(rows_with_imagery) > 0:
    print("\nRows with imagery:")
    for _, row in rows_with_imagery.iterrows():
        print(
            f"  Row {row['row_index']}: {row['items_found']} items ({row['datetime_range']})"
        )

=== SEARCHING SENTINEL-1 RTC FOR ALL EMDAT ROWS ===


Processing rows:   0%|          | 0/43 [00:00<?, ?it/s]

Filtered by 7 adm1 names: ['Catamarca', 'Cordoba', 'Jujuy', 'La Rioja', 'Salta', 'Santiago Del Estero', 'Tucuman']
Bounding box coordinates: {'min_lat': -30.120678, 'max_lat': -21.805624, 'min_lon': -69.114097, 'max_lon': -61.627238}
Bounding box area: 62.253637


Processing rows:   2%|▏         | 1/43 [00:00<00:13,  3.14it/s]

Filtered by 8 adm2 names: ['Avellaneda', 'Berisso', 'Lanus', 'Quilmes', 'San Miguel', 'San  Fernando', 'Parana', 'Rosario']
Bounding box coordinates: {'min_lat': -39.92531, 'max_lat': -27.468575, 'min_lon': -67.8722, 'max_lon': -56.9436}
Bounding box area: 136.134674


...

Filtered by 1 adm1 names: ['Catamarca']
Bounding box coordinates: {'min_lat': -30.116125, 'max_lat': -25.129197, 'min_lon': -69.114097, 'max_lon': -64.839508}
Bounding box area: 21.317068


Processing rows:  91%|█████████ | 39/43 [00:23<00:02,  1.86it/s]

Filtered by 1 adm2 names: ['Quilmes']
Bounding box coordinates: {'min_lat': -34.801014, 'max_lat': -34.678736, 'min_lon': -58.346916, 'max_lon': -58.194752}
Bounding box area: 0.018606


Processing rows: 100%|██████████| 43/43 [00:24<00:00,  1.79it/s]

No admin names found!
No admin names found!
No admin names found!

Total rows: 43
Rows with imagery: 16
Total items found: 211

Rows with imagery:
  Row 22: 2 items (2014-11-04/2014-11-06)
  Row 23: 21 items (2015-03-04/2015-03-06)
  Row 24: 5 items (2015-08-12/2015-08-14)
  Row 25: 6 items (2016-01-11/2016-01-13)
  Row 26: 19 items (2016-04-15/2016-04-17)
  Row 27: 26 items (2016-12-26/2016-12-28)
  Row 28: 48 items (2017-04-07/2017-04-09)
  Row 29: 5 items (2017-06-14/2017-06-16)
  Row 31: 16 items (2018-02-21/2018-02-23)
  Row 33: 3 items (2018-11-12/2018-11-14)
  Row 34: 20 items (2019-01-17/2019-01-19)
  Row 35: 12 items (2019-04-23/2019-04-25)
  Row 36: 3 items (2019-05-20/2019-05-22)
  Row 37: 13 items (2020-02-19/2020-02-21)
  Row 38: 11 items (2021-03-01/2021-03-03)
  Row 39: 1 items (2023-07-05/2023-07-07)


In [11]:
import pandas as pd
from tqdm import tqdm

# Initialize results storage
results = []

print("=== SEARCHING SENTINEL-2 L2A FOR ALL EMDAT ROWS ===")

# Process each row
for idx, row in tqdm(emdat.iterrows(), total=len(emdat), desc="Processing rows"):
    try:
        # Get bounding box for this row
        bbox_geometry, bbox_dict = get_flood_bounding_box(row, "ARG")

        if bbox_geometry is None:
            results.append(
                {
                    "row_index": idx,
                    "event_name": row.get("Event Name", "Unknown"),
                    "end_date": row.get("end_date", None),
                    "datetime_range": row.get("datetime_range", None),
                    "bbox_available": False,
                    "items_found": 0,
                    "error": "No bounding box available",
                }
            )
            continue

        # Convert shapely geometry bounds to bbox format
        bounds = bbox_geometry.bounds
        bbox = [bounds[0], bounds[1], bounds[2], bounds[3]]

        # Get datetime range
        datetime_range = row.get("datetime_range")

        if datetime_range is None:
            results.append(
                {
                    "row_index": idx,
                    "event_name": row.get("Event Name", "Unknown"),
                    "end_date": row.get("end_date", None),
                    "datetime_range": None,
                    "bbox_available": True,
                    "items_found": 0,
                    "error": "No valid date range",
                }
            )
            continue

        # Search for Sentinel-2 L2A data with cloud cover filtering
        search = catalog.search(
            collections=["sentinel-2-l2a"],
            bbox=bbox,
            datetime=datetime_range,
            query={
                "eo:cloud_cover": {"lt": 20},  # Less than 20% cloud cover
                "platform": {"in": ["sentinel-2a", "sentinel-2b"]},
            },
        )

        items = search.item_collection()

        # Store results
        results.append(
            {
                "row_index": idx,
                "event_name": row.get("Event Name", "Unknown"),
                "end_date": row.get("end_date", None),
                "datetime_range": datetime_range,
                "bbox_available": True,
                "items_found": len(items),
                "bbox_coords": bbox,
                "error": None,
            }
        )

    except Exception as e:
        results.append(
            {
                "row_index": idx,
                "event_name": row.get("Event Name", "Unknown"),
                "end_date": row.get("end_date", None),
                "datetime_range": row.get("datetime_range", None),
                "bbox_available": False,
                "items_found": 0,
                "error": str(e),
            }
        )

# Convert results to DataFrame
results_df = pd.DataFrame(results)

# Summary statistics
print(f"\nTotal rows: {len(results_df)}")
print(f"Rows with imagery: {(results_df['items_found'] > 0).sum()}")
print(f"Total items found: {results_df['items_found'].sum()}")

=== SEARCHING SENTINEL-2 L2A FOR ALL EMDAT ROWS ===


Processing rows:   0%|          | 0/43 [00:00<?, ?it/s]

Filtered by 7 adm1 names: ['Catamarca', 'Cordoba', 'Jujuy', 'La Rioja', 'Salta', 'Santiago Del Estero', 'Tucuman']
Bounding box coordinates: {'min_lat': -30.120678, 'max_lat': -21.805624, 'min_lon': -69.114097, 'max_lon': -61.627238}
Bounding box area: 62.253637


Processing rows:   2%|▏         | 1/43 [00:00<00:13,  3.21it/s]

Filtered by 8 adm2 names: ['Avellaneda', 'Berisso', 'Lanus', 'Quilmes', 'San Miguel', 'San  Fernando', 'Parana', 'Rosario']
Bounding box coordinates: {'min_lat': -39.92531, 'max_lat': -27.468575, 'min_lon': -67.8722, 'max_lon': -56.9436}
Bounding box area: 136.134674


...

Filtered by 1 adm1 names: ['Catamarca']
Bounding box coordinates: {'min_lat': -30.116125, 'max_lat': -25.129197, 'min_lon': -69.114097, 'max_lon': -64.839508}
Bounding box area: 21.317068


Processing rows:  91%|█████████ | 39/43 [00:20<00:01,  2.11it/s]

Filtered by 1 adm2 names: ['Quilmes']
Bounding box coordinates: {'min_lat': -34.801014, 'max_lat': -34.678736, 'min_lon': -58.346916, 'max_lon': -58.194752}
Bounding box area: 0.018606


Processing rows: 100%|██████████| 43/43 [00:20<00:00,  2.05it/s]

No admin names found!
No admin names found!
No admin names found!

Total rows: 43
Rows with imagery: 0
Total items found: 0


In [ ]:
# Show rows with imagery
rows_with_imagery = results_df[results_df["items_found"] > 0]
if len(rows_with_imagery) > 0:
    print("\nRows with imagery:")
    for _, row in rows_with_imagery.iterrows():
        print(
            f"  Row {row['row_index']}: {row['items_found']} items ({row['datetime_range']})"
        )

In [ ]:
# Step 3: Preview Items (Optional)
item_id = {(i, item.id): i for i, item in enumerate(items)}
item_sel = pn.widgets.Select(value=0, options=item_id, name="Select Item")


def get_preview(i):
    return pn.panel(items[i].assets["rendered_preview"].href, height=300)


pn.Row(item_sel, pn.bind(get_preview, item_sel)).servable()